In [1]:
# -*- coding: utf-8 -*-
"""
蛋白质序列标签预测 - ProtVec 预测脚本

本脚本使用训练好的ProtVec模型和分类器对测试数据进行预测，并生成提交结果。
"""

import pickle
import gensim
import gensim.models
import os
import sys
import numpy as np
import pandas as pd
from joblib import load


def generate_ngrams(sequence, n=3):
    """生成蛋白质序列的n-gram"""
    return [sequence[i:i+n] for i in range(len(sequence)-n+1)]


def extract_features_for_position(sequence, position, model, n=3):
    """为序列中的特定位置提取特征"""
    # 获取包含该位置的所有n-gram
    relevant_ngrams = []
    for i in range(max(0, position-n+1), min(len(sequence)-n+1, position+1)):
        ngram = sequence[i:i+n]
        if ngram in model.wv:
            relevant_ngrams.append(ngram)
    
    # 如果没有相关n-gram，使用随机向量
    if not relevant_ngrams:
        return np.random.randn(model.vector_size)
    
    # 计算所有相关n-gram向量的平均值
    return np.mean([model.wv[ngram] for ngram in relevant_ngrams], axis=0)

path = "../Data/WSAA_data_public.pkl"
"""使用训练好的ProtVec模型和分类器对测试数据进行预测"""
# 加载模型
print("加载模型...")
model_protvec = gensim.models.Word2Vec.load("protvec_model.w2v")
classifier = load("classifier_model.joblib")

# 加载测试数据
print("加载测试数据...")
test_datas = pickle.load(open(path, "rb"))

# 预测结果
print("开始预测...")
submit_data = []

for i, data in enumerate(test_datas):
    if i % 10 == 0:
        # print(f"处理第 {i+1}/{len(test_datas)} 个序列...")
    
    sequence = data["sequence"]
    
    # 为序列中的每个位置提取特征并预测
    data_x = []
    for idx in range(len(sequence)):
        features = extract_features_for_position(sequence, idx, model_protvec)
        data_x.append(features)
    
    # 转换为numpy数组并预测
    data_x = np.array(data_x)
    pred = classifier.predict(data_x)
    
    # 转换为字符串
    pred_str = "".join([str(int(p)) for p in pred])
    
    # 添加到提交数据
    submit_data.append([data['id'], data['sequence'], pred_str])

# 创建提交DataFrame
submit_df = pd.DataFrame(submit_data)
submit_df.columns = ["proteinID", "sequence", "IDRs"]

加载模型...
加载测试数据...
开始预测...
处理第 1/2179 个序列...
处理第 11/2179 个序列...
处理第 21/2179 个序列...
处理第 31/2179 个序列...
处理第 41/2179 个序列...
处理第 51/2179 个序列...
处理第 61/2179 个序列...
处理第 71/2179 个序列...
处理第 81/2179 个序列...
处理第 91/2179 个序列...
处理第 101/2179 个序列...
处理第 111/2179 个序列...
处理第 121/2179 个序列...
处理第 131/2179 个序列...
处理第 141/2179 个序列...
处理第 151/2179 个序列...
处理第 161/2179 个序列...
处理第 171/2179 个序列...
处理第 181/2179 个序列...
处理第 191/2179 个序列...
处理第 201/2179 个序列...
处理第 211/2179 个序列...
处理第 221/2179 个序列...
处理第 231/2179 个序列...
处理第 241/2179 个序列...
处理第 251/2179 个序列...
处理第 261/2179 个序列...
处理第 271/2179 个序列...
处理第 281/2179 个序列...
处理第 291/2179 个序列...
处理第 301/2179 个序列...
处理第 311/2179 个序列...
处理第 321/2179 个序列...
处理第 331/2179 个序列...
处理第 341/2179 个序列...
处理第 351/2179 个序列...
处理第 361/2179 个序列...
处理第 371/2179 个序列...
处理第 381/2179 个序列...
处理第 391/2179 个序列...
处理第 401/2179 个序列...
处理第 411/2179 个序列...
处理第 421/2179 个序列...
处理第 431/2179 个序列...
处理第 441/2179 个序列...
处理第 451/2179 个序列...
处理第 461/2179 个序列...
处理第 471/2179 个序列...
处理第 481/2179 个序列...
处理第 4

In [2]:
submit_df

,proteinID,sequence,IDRs
0,disordered_protein_0,MKQFGLAAFDELKDGKYNDVNKTILEKQSVELRDQLMVFQERLVEF...,0000001100110100000000000110001000000000000000...
1,disordered_protein_1,MQAIKCVVVGDGAVGKTCLLISYTTNAFPGEYIPTVFDNYSANVMV...,0000000000001100000000000000110000000000000000...
2,disordered_protein_2,MSTARTENPVIMGLSSQNGQLRGPVKPTGGPGGGGTQTQQQMNQLK...,0101000000000111100000011111111111100011000000...
3,disordered_protein_3,MWRCGGRQGLCVLRRLSGGHAHHRAWRWNSNRACERALQYKLGDKI...,0000000000000011110000000000000000000000000000...
4,disordered_protein_4,MEAQSHSSTTTEKKKVENSIVKCSTRTDVSEKAVASSTTSNEDESP...,0110011111111111000000000000011111111111011111...
...,...,...,...
2174,disordered_protein_2174,MEPSSPQDEGLRKKQPKKPLPEVLPRPPRALFCLTLQNPLRKACIS...,0111111010001111111111101111100000000000000000...
2175,disordered_protein_2175,MSRVTVLQSQLPAYNRLKTPYESELIATVKKLTTPGKGLLAADESI...,0000000011110000000001110000011011111101111100...
2176,disordered_protein_2176,MNAAGGGSGAQAAAVAAGNNSLSHNALLSTASGATTMPMAQLADGW...,0011111111111111100011000001111111000000110000...
2177,disordered_protein_2177,MKLWDVVAVCLVLLHTASAFPLPAGKRPPEAPAEDRSLGRRRAPFA...,0000000000000000111001111111111111110001111000...
